In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy.fft import fft, fftfreq
import tensorflow as tf
from tensorflow.keras import layers, models

from sklearn.decomposition import PCA


In [2]:
#For importing stuff outside of this notebook's directory
%load_ext autoreload
%autoreload 2
import sys 
sys.path.insert(0, "../")

import midiio
import frequency_analysis as frqa


# Exploring and preparing the dataset

In [3]:
# Begin by opening all files in the data folder

notes_database = [] 

for filename in os.listdir(os.path.join(os.getcwd(), "..\\data\\notes")):
    note = filename.replace(".mp3", "")
    data, sr = midiio.read_file("../data/notes/" + filename)
    
    _, power = frqa.get_frequencies(data, sr)
    # L/R support
    notes_database.append((note, *np.abs(power[0]), *np.abs(power[1])))

notes_database = pd.DataFrame(notes_database)
notes_database

c:\Users\JOAQUIN DELA ROSA\anaconda3\envs\tfgpu\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


,0,1,2,3,4,5,6,7,8,9,...,505,506,507,508,509,510,511,512,513,514
0,A0,1.832628e-06,2.991571e-06,1.595974e-06,5.456926e-07,1.522986e-07,6.778212e-08,3.820701e-08,2.554736e-08,2.089292e-08,...,3.335342e-15,3.299133e-15,3.400284e-15,3.537889e-15,3.551728e-15,3.435660e-15,3.368344e-15,3.443583e-15,3.575597e-15,1.819026e-15
1,A1,1.655228e-07,3.677820e-07,3.715728e-07,2.565321e-07,1.256248e-07,6.494530e-08,4.947334e-08,3.893961e-08,2.467447e-08,...,3.640973e-15,3.534595e-15,3.402364e-15,3.337021e-15,3.372872e-15,3.467242e-15,3.546236e-15,3.565701e-15,3.554374e-15,1.774819e-15
2,A2,3.203580e-07,6.332684e-07,5.081856e-07,2.707909e-07,1.151136e-07,7.088545e-08,5.391019e-08,3.816959e-08,2.466852e-08,...,3.466295e-15,3.598625e-15,3.608903e-15,3.545568e-15,3.502057e-15,3.482703e-15,3.560364e-15,3.689894e-15,3.666573e-15,1.797142e-15
3,A3,3.816294e-08,1.838512e-07,3.695091e-07,3.840436e-07,2.131007e-07,6.465614e-08,2.076840e-08,1.798612e-08,1.610965e-08,...,3.433484e-15,3.511976e-15,3.451108e-15,3.395139e-15,3.427414e-15,3.506119e-15,3.569516e-15,3.570959e-15,3.520368e-15,1.744540e-15
4,A4,1.584768e-09,2.830152e-09,1.114473e-08,8.979988e-08,2.781770e-07,4.136571e-07,3.201396e-07,1.217145e-07,1.695874e-08,...,3.439789e-15,3.378218e-15,3.419274e-15,3.504837e-15,3.530532e-15,3.463896e-15,3.334923e-15,3.227481e-15,3.232594e-15,1.635116e-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Gb3,9.687317e-08,3.232219e-07,5.261576e-07,5.776808e-07,4.940960e-07,3.349306e-07,1.560783e-07,4.847950e-08,1.701218e-08,...,3.477329e-15,3.503925e-15,3.549178e-15,3.629333e-15,3.659649e-15,3.587052e-15,3.519816e-15,3.598196e-15,3.774881e-15,1.932609e-15
85,Gb4,1.352648e-09,4.164893e-09,2.462370e-08,8.502476e-08,1.426371e-07,1.249823e-07,5.836280e-08,2.571078e-08,3.329815e-08,...,3.679358e-15,3.724090e-15,3.748381e-15,3.730107e-15,3.650519e-15,3.534291e-15,3.467770e-15,3.425926e-15,3.290698e-15,1.595635e-15
86,Gb5,6.210749e-10,1.463880e-09,1.532267e-09,9.530066e-10,3.939673e-10,4.429340e-10,2.079226e-09,7.836507e-09,1.559274e-08,...,3.465915e-15,3.572245e-15,3.620074e-15,3.557567e-15,3.543169e-15,3.607771e-15,3.620712e-15,3.554486e-15,3.484178e-15,1.728787e-15
87,Gb6,3.194586e-10,6.738839e-10,6.220580e-10,3.951775e-10,1.852742e-10,9.394682e-11,6.207131e-11,4.069380e-11,2.356722e-11,...,3.538932e-15,3.474596e-15,3.469651e-15,3.455798e-15,3.446382e-15,3.446222e-15,3.423163e-15,3.406330e-15,3.448624e-15,1.743454e-15


In [4]:
# Preprocessing steps. Drop columns that have only NaNs in them
notes_database.dropna('columns', inplace=True)

# Reindex to use column0
notes_database.set_index(0, inplace=True)

C:\Users\JOAQUIN DELA ROSA\AppData\Local\Temp\ipykernel_21076\2077169024.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  notes_database.dropna('columns', inplace=True)


# Pitch embedding

In [5]:
notes_database

,1,2,3,4,5,6,7,8,9,10,...,505,506,507,508,509,510,511,512,513,514
0,,,,,,,,,,,,,,,,,,,,,
A0,1.832628e-06,2.991571e-06,1.595974e-06,5.456926e-07,1.522986e-07,6.778212e-08,3.820701e-08,2.554736e-08,2.089292e-08,1.684670e-08,...,3.335342e-15,3.299133e-15,3.400284e-15,3.537889e-15,3.551728e-15,3.435660e-15,3.368344e-15,3.443583e-15,3.575597e-15,1.819026e-15
A1,1.655228e-07,3.677820e-07,3.715728e-07,2.565321e-07,1.256248e-07,6.494530e-08,4.947334e-08,3.893961e-08,2.467447e-08,1.326909e-08,...,3.640973e-15,3.534595e-15,3.402364e-15,3.337021e-15,3.372872e-15,3.467242e-15,3.546236e-15,3.565701e-15,3.554374e-15,1.774819e-15
A2,3.203580e-07,6.332684e-07,5.081856e-07,2.707909e-07,1.151136e-07,7.088545e-08,5.391019e-08,3.816959e-08,2.466852e-08,1.336676e-08,...,3.466295e-15,3.598625e-15,3.608903e-15,3.545568e-15,3.502057e-15,3.482703e-15,3.560364e-15,3.689894e-15,3.666573e-15,1.797142e-15
A3,3.816294e-08,1.838512e-07,3.695091e-07,3.840436e-07,2.131007e-07,6.465614e-08,2.076840e-08,1.798612e-08,1.610965e-08,1.109537e-08,...,3.433484e-15,3.511976e-15,3.451108e-15,3.395139e-15,3.427414e-15,3.506119e-15,3.569516e-15,3.570959e-15,3.520368e-15,1.744540e-15
A4,1.584768e-09,2.830152e-09,1.114473e-08,8.979988e-08,2.781770e-07,4.136571e-07,3.201396e-07,1.217145e-07,1.695874e-08,6.551337e-10,...,3.439789e-15,3.378218e-15,3.419274e-15,3.504837e-15,3.530532e-15,3.463896e-15,3.334923e-15,3.227481e-15,3.232594e-15,1.635116e-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gb3,9.687317e-08,3.232219e-07,5.261576e-07,5.776808e-07,4.940960e-07,3.349306e-07,1.560783e-07,4.847950e-08,1.701218e-08,9.460430e-09,...,3.477329e-15,3.503925e-15,3.549178e-15,3.629333e-15,3.659649e-15,3.587052e-15,3.519816e-15,3.598196e-15,3.774881e-15,1.932609e-15
Gb4,1.352648e-09,4.164893e-09,2.462370e-08,8.502476e-08,1.426371e-07,1.249823e-07,5.836280e-08,2.571078e-08,3.329815e-08,3.513385e-08,...,3.679358e-15,3.724090e-15,3.748381e-15,3.730107e-15,3.650519e-15,3.534291e-15,3.467770e-15,3.425926e-15,3.290698e-15,1.595635e-15
Gb5,6.210749e-10,1.463880e-09,1.532267e-09,9.530066e-10,3.939673e-10,4.429340e-10,2.079226e-09,7.836507e-09,1.559274e-08,1.659913e-08,...,3.465915e-15,3.572245e-15,3.620074e-15,3.557567e-15,3.543169e-15,3.607771e-15,3.620712e-15,3.554486e-15,3.484178e-15,1.728787e-15


In [6]:
# We do this using PCA to reduce the dimensionality of the data.

pca = PCA(n_components=0.99)
note_embedding = pca.fit_transform(notes_database)

pca.explained_variance_

array([8.6323380e-13, 9.7927803e-14, 2.2365417e-14, 1.6211535e-14,
       9.3714539e-15, 4.2263916e-15, 3.4568215e-15, 3.2110750e-15,
       1.9564329e-15], dtype=float32)

In [7]:
#Convert it to a dataframe and then display it.

note_embedding = pd.DataFrame(note_embedding)
note_embedding.set_index(notes_database.index, inplace=True)
note_embedding

,0,1,2,3,4,5,6,7,8
0,,,,,,,,,
A0,3.540292e-06,-4.115869e-07,6.716873e-08,-1.458114e-07,-1.268204e-07,7.939543e-08,1.378082e-08,-6.705671e-08,-5.689540e-09
A1,8.173296e-08,1.337368e-07,1.026474e-07,3.516261e-08,-7.112271e-09,-4.278378e-08,-2.250895e-08,1.797436e-08,1.590730e-08
A2,5.277272e-07,1.828374e-07,2.927341e-07,2.637224e-08,-1.762827e-07,-8.824146e-08,1.510990e-09,5.115346e-08,4.748171e-08
A3,-1.875526e-07,2.138749e-07,-1.764050e-07,3.222972e-08,2.523912e-07,-4.130912e-08,-2.884339e-08,2.705423e-08,7.624455e-08
A4,-5.446852e-07,1.905351e-07,-3.562681e-07,-2.531930e-07,-2.012016e-07,3.795784e-08,2.541828e-08,-4.938138e-09,1.389234e-08
...,...,...,...,...,...,...,...,...,...
Gb3,1.996507e-07,6.667919e-07,-1.197080e-07,2.110420e-07,-1.072153e-07,-2.799336e-08,-8.633688e-09,5.729826e-09,-3.011173e-08
Gb4,-5.572599e-07,-4.393588e-08,-8.878970e-08,1.385737e-08,-3.709946e-08,3.883821e-08,1.011115e-08,-5.102035e-08,-8.401251e-09
Gb5,-6.021381e-07,-2.280674e-07,2.590371e-08,2.046038e-08,1.631872e-08,-3.907721e-08,-1.726122e-08,-2.253885e-08,-5.988089e-08


In [8]:
# We can also get the raw embedding matrix as follows.
note_embedding_matrix = pd.DataFrame(pca.components_)
note_embedding_matrix

,0,1,2,3,4,5,6,7,8,9,...,504,505,506,507,508,509,510,511,512,513
0,0.411577,0.705570,0.428735,0.173545,0.049232,0.012501,0.002764,0.002633,0.003607,0.002424,...,6.847698e-11,9.061268e-11,1.376952e-10,1.709041e-10,1.804019e-10,1.687217e-10,1.468734e-10,1.401901e-10,1.567325e-10,8.457093e-11
1,-0.223686,-0.255616,0.144036,0.417537,0.415148,0.253055,0.104774,0.031144,0.004687,-0.005240,...,3.342391e-10,3.941135e-10,4.727812e-10,5.204683e-10,5.464617e-10,5.431234e-10,5.085149e-10,4.876861e-10,5.136537e-10,2.690463e-10
2,-0.038001,-0.093358,-0.130118,-0.215767,-0.359600,-0.408247,-0.278197,-0.115481,-0.036624,-0.012260,...,-2.633485e-10,-1.870913e-10,-1.841808e-10,-2.403145e-10,-2.503381e-10,-2.036986e-10,-1.505842e-10,-1.149030e-10,-1.145342e-10,-6.309141e-11
3,-0.194990,-0.144875,0.271545,0.400141,0.178866,-0.141609,-0.272270,-0.180889,-0.056403,-0.001283,...,-2.358648e-10,-2.669214e-10,-4.080187e-10,-4.692747e-10,-4.061895e-10,-3.122370e-10,-2.772868e-10,-3.222293e-10,-3.996679e-10,-2.182802e-10
4,-0.051942,0.059114,0.317173,0.233113,-0.181756,-0.423667,-0.302087,-0.095404,0.000506,0.017531,...,-1.902866e-10,-1.465231e-10,-1.456977e-10,-2.271744e-10,-2.819703e-10,-2.254801e-10,-1.361355e-10,-9.956348e-11,-8.513382e-11,-3.734665e-11
5,0.187511,0.112211,-0.212538,-0.088190,0.190198,0.147403,-0.131495,-0.258718,-0.168051,-0.062887,...,2.746335e-10,1.879842e-10,2.604123e-11,-1.723867e-10,-1.204658e-10,2.027529e-10,4.393959e-10,4.895781e-10,5.554452e-10,3.083553e-10
6,0.077419,0.053008,-0.099787,-0.079667,0.062325,0.092009,-0.023370,-0.103061,-0.078935,-0.037335,...,-8.290159e-10,-1.329598e-09,-1.582782e-09,-1.509911e-09,-1.254135e-09,-1.127473e-09,-1.190487e-09,-1.097588e-09,-7.018489e-10,-2.248380e-10
7,-0.150021,-0.071569,0.192033,0.052585,-0.171664,-0.078985,0.162161,0.210772,0.078633,-0.040570,...,1.147787e-09,1.255295e-09,1.899569e-09,2.559903e-09,3.080463e-09,3.409911e-09,3.517486e-09,3.624844e-09,3.922171e-09,2.058907e-09
8,-0.107136,-0.067568,0.104124,0.023982,-0.087004,-0.039142,-0.009625,-0.073914,-0.119924,-0.113835,...,1.427892e-09,3.401514e-10,1.625064e-11,5.323144e-10,1.089127e-09,9.670702e-10,3.259397e-10,-2.478202e-10,-4.877624e-10,-2.619015e-10


In [9]:
# The embedding matrix will be important so let us save it
note_embedding_matrix.to_csv("../models/note_embedding.csv")

# Validation

In [10]:

test_database = [] 
path = "../data/notes-validation/"

for filename in os.listdir(os.path.join(os.getcwd(), path)):
    note = filename.replace(".mp3", "")
    data, sr = midiio.read_file(path + filename)

    res = frqa.get_frequencies(data, sr)
    power = res[1]
    # L/R support
    test_database.append((note, *np.abs(power[0]), *np.abs(power[1])))

test_database = pd.DataFrame(test_database)
test_database

c:\Users\JOAQUIN DELA ROSA\anaconda3\envs\tfgpu\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100
44100


,0,1,2,3,4,5,6,7,8,9,...,505,506,507,508,509,510,511,512,513,514
0,a-3,2.373361e-08,6.571376e-08,1.179157e-07,1.871855e-07,2.181240e-07,1.684512e-07,8.749048e-08,4.522309e-08,4.338744e-08,...,3.705571e-15,3.597869e-15,3.409628e-15,3.141904e-15,3.023313e-15,3.117207e-15,3.303508e-15,3.382900e-15,3.241404e-15,1.559366e-15
1,a-4,1.377239e-08,7.398746e-08,1.673488e-07,2.097492e-07,1.884989e-07,1.675246e-07,1.374612e-07,8.244061e-08,4.344242e-08,...,3.377904e-15,3.526933e-15,3.626170e-15,3.548381e-15,3.401931e-15,3.366075e-15,3.406998e-15,3.469982e-15,3.572275e-15,1.816452e-15
2,a-5,1.078053e-09,2.306629e-09,4.519868e-09,5.145461e-08,2.127653e-07,3.958799e-07,3.800033e-07,1.864061e-07,3.989643e-08,...,3.495476e-15,3.412141e-15,3.341259e-15,3.384747e-15,3.392938e-15,3.307220e-15,3.325708e-15,3.527928e-15,3.792556e-15,1.959921e-15
3,a3,3.308904e-08,8.740874e-08,1.431529e-07,2.018192e-07,2.077214e-07,1.440936e-07,7.281714e-08,5.046423e-08,6.272588e-08,...,3.287507e-15,3.232368e-15,3.078277e-15,2.887539e-15,2.866602e-15,3.102179e-15,3.467435e-15,3.645530e-15,3.502857e-15,1.683992e-15
4,a4,5.732811e-09,2.384967e-08,5.250872e-08,1.172765e-07,2.494538e-07,3.461103e-07,2.747436e-07,1.221768e-07,3.949842e-08,...,3.413590e-15,3.648920e-15,3.724993e-15,3.669485e-15,3.594450e-15,3.517830e-15,3.499763e-15,3.569770e-15,3.625601e-15,1.816170e-15
5,a5,1.245263e-09,1.740248e-09,1.457726e-08,1.421390e-07,4.505227e-07,6.754179e-07,5.270127e-07,2.045454e-07,3.403952e-08,...,3.374901e-15,3.288888e-15,3.278254e-15,3.306363e-15,3.334908e-15,3.345756e-15,3.396867e-15,3.523231e-15,3.604643e-15,1.805721e-15
6,b3,9.609874e-09,6.583577e-08,1.700600e-07,2.239129e-07,1.793558e-07,1.225225e-07,9.432458e-08,6.315300e-08,3.684132e-08,...,3.249050e-15,3.444158e-15,3.557009e-15,3.554370e-15,3.543997e-15,3.587179e-15,3.579673e-15,3.488797e-15,3.501717e-15,1.781762e-15
7,b4,9.653339e-09,6.594441e-08,1.702213e-07,2.239932e-07,1.788476e-07,1.211219e-07,9.273202e-08,6.231646e-08,3.681346e-08,...,3.578447e-15,3.384362e-15,3.256035e-15,3.231916e-15,3.232946e-15,3.250134e-15,3.416264e-15,3.636892e-15,3.668826e-15,1.809108e-15
8,b5,8.062782e-10,2.027912e-09,1.901745e-09,2.035084e-08,1.199555e-07,2.844127e-07,3.375069e-07,2.085583e-07,6.069861e-08,...,3.458563e-15,3.354950e-15,3.304373e-15,3.386442e-15,3.408152e-15,3.281896e-15,3.215817e-15,3.292923e-15,3.381760e-15,1.704975e-15
9,c-3,1.201222e-08,6.951450e-08,2.099790e-07,3.685232e-07,4.082404e-07,3.173968e-07,1.907240e-07,9.272283e-08,3.601000e-08,...,3.116384e-15,3.249506e-15,3.557941e-15,3.676784e-15,3.658992e-15,3.717152e-15,3.731167e-15,3.637134e-15,3.565172e-15,1.775129e-15
